In [2]:
import os
import tarfile
import urllib.request as url
import numpy as np
import tensorflow as tf

# download cifar
check if CIFAR in directory, if not, download 

In [3]:
def download_data():
    if not os.path.exists('.././data'):
        os.mkdir('.././data')
    if not os.path.exists('.././data/cifar-10-python.tar.gz'):
        print("CIFAR-10 not found, downloading...")
        url.urlretrieve("https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz",
                        ".././data/cifar-10-python.tar.gz")
        print("done")
        print("Download completed!")
    else:
        print('CIFAR-10 already exists.')
        
# download_data()

In [4]:
import glob
import _pickle as pickle
def load_data():
    if not os.path.exists('.././data/cifar-10-python.tar.gz'):
        download_data()
    if not os.path.exists('.././data/cifar-10-batches-py/'):
        print("extracting...")
        package = tarfile.open('.././data/cifar-10-python.tar.gz')
        package.extractall('.././data')
        package.close()
        
    root_dir = os.getcwd()
    os.chdir('.././data/cifar-10-batches-py')
    train_data = []
    train_label = []
    test_data = []
    test_label = []
    data_train = glob.glob('data_batch*')
    print(data_train)
    #try:
    for name in data_train:
        handle = open(name, 'rb')
        cmap = pickle.load(handle, encoding='bytes')
        train_data.append(cmap[b'data'])
        train_label.append(cmap[b'labels'])
        handle.close()
    # Turn the dataset into numpy compatible arrays.
    train_data = np.concatenate(train_data, axis=0)
    train_label = np.concatenate(train_label)
    handle = open('test_batch', 'rb')
    cmap = pickle.load(handle, encoding='bytes')
    test_data.append(cmap[b'data'])
    test_label.append(cmap[b'labels'])
    test_data = np.array(test_data[0])
    test_label = np.array(test_label[0])
    #except BaseException:
#         os.chdir(root_dir)
#         print('Something went wrong...')
#         return None
    os.chdir(root_dir)
    return train_data,train_label,test_data,test_label

train_data,train_label,test_data,test_label = load_data()
val_size = 1000
val_data = train_data[-1000:]
val_label = train_label[-1000:]
train_data = train_data[:-1000]
train_label = train_label [:-1000]
print(train_data.shape, train_label.shape)
print(val_data.shape, val_label.shape)
print(test_data.shape, test_label.shape)

['data_batch_1', 'data_batch_4', 'data_batch_3', 'data_batch_2', 'data_batch_5']
(49000, 3072) (49000,)
(1000, 3072) (1000,)
(10000, 3072) (10000,)


# build neural network using tensorflow


In [17]:
import tensorflow as tf
def cnn_4_cifar(features, labels, mode = tf.estimator.ModeKeys.TRAIN):
    # features aka. x, labels aka y, i don't know mode right now
#     train_x = features
#     train_y = labels
#     input_data = tf.placeholder(
#         dtype = tf.float32,
#         shape = [None, 3072],
#         name = "input")
    
#     labels = tf.placeholder(
#         dtype = tf.int32,
#         shape = [None, ],
#         name = "labels")
    
    input_data = features["x"]
    
   
    
    ###### normalize layer ######
    normal_layer = input_data/255.
    
#     normal_layer = tf.layers.batch_normalization(
#         inputs = input_data,
#         training= mode == tf.estimator.ModeKeys.TRAIN,
#         reuse=None,
#         name = 'batch_normal'
#     )
    

    #normalize_layer = tf.nn.l2_normalize()
    
    input_labels = tf.one_hot(indices=labels,
                        depth = 10,
                        name='one_hot_label',
                        axis=1,
                       dtype = tf.int32)
    
    ###### input layer ######
    
    reshape_layer = tf.reshape(normal_layer, [-1,3, 32, 32])
    reshape_layer = tf.transpose(reshape_layer, perm=[0,2,3,1])
    reshape_layer = tf.cast(reshape_layer, dtype = tf.float32)
    
    
    ###### convolutional layer #1 ######
    
    conv1 = tf.layers.conv2d(
        inputs = reshape_layer,
        filters = 16,
        kernel_size = [5,5],
        padding = "valid",
        activation = tf.nn.relu
    )
    
    ###### pooling layer #1 ######
    
    pool1 = tf.layers.max_pooling2d(
        inputs=conv1, pool_size = [2,2], strides = 2)
    
    ###### lrn layer #1 #####
    
    lrn1 = tf.nn.local_response_normalization(
        pool1,
        depth_radius = 5
)
    
    ###### convolutional layer #2 ######
        
    conv2 = tf.layers.conv2d(
        inputs = lrn1,
        filters = 64,
        kernel_size = [5,5],
        padding = "valid",
        activation = tf.nn.relu
    )
    
    ###### lrn layer #2 #####
    
    lrn2 = tf.nn.local_response_normalization(
        conv2,
        depth_radius = 5
    )
    
    ###### pooling layer #2 ######
    
    pool2 = tf.layers.max_pooling2d(
        inputs = lrn2,
        pool_size = [2,2],
        strides = 2)
    
    conv3 = tf.layers.conv2d(
        inputs = pool2,
        filters = 32,
        kernel_size = [1,1],
        padding = "valid",
        activation = tf.nn.relu
    )

    
    ###### flat layer ######
    
    pool_flat = tf.reshape(conv3, [-1, 5 * 5 * 32])
    
    ###### dense layer ######
    
    dense1 = tf.layers.dense(
        inputs = pool_flat,                     
        units = 230,                     
        activation = tf.nn.relu
    )
    
    dense_normal1 = tf.layers.batch_normalization(dense1)
    
    dense2 = tf.layers.dense(
        inputs = dense_normal1,                     
        units = 42,                     
        activation = tf.nn.relu
    )
    
    dense3 = tf.layers.dense(
        inputs = dense2,                     
        units = 80,                     
        activation = tf.nn.relu
    )
    
    ###### dropout layer ######
    
    dropout = tf.layers.dropout(
        inputs = dense3,
        rate = 0.1,
        training = (mode == tf.estimator.ModeKeys.TRAIN)
    )
    ###### logits layer ######
    
    logits = tf.layers.dense(inputs=dropout, units=10)
    
    ###### prediction ######
    
    predictions = {
        "class" : tf.argmax(input = logits, axis = 1),
        "prob" : tf.nn.softmax(
            logits, 
            name = "softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(
            mode=mode, 
            predictions = predictions)
    
    ###### loss ######
    
    loss = tf.losses.sparse_softmax_cross_entropy(
        labels = labels,
        logits = logits
    )
    
    ##### train op ######
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.RMSPropOptimizer(
            learning_rate = 0.0001
        )
#         optimizer = tf.train.GradientDescentOptimizer(
#             learning_rate = 0.01
#         )
        train_op = optimizer.minimize(
            loss = loss,
            global_step = tf.train.get_global_step()
        )
        return tf.estimator.EstimatorSpec(
            mode = mode,
            loss = loss,
            train_op = train_op
            )
    
    ##### eval op ######
    
    if mode == tf.estimator.ModeKeys.EVAL:
        eval_metric_ops = {
            "accuracy": tf.metrics.accuracy(
                labels=labels,
                predictions=predictions["class"])}
    return tf.estimator.EstimatorSpec(
        mode=mode,
        loss=loss,
        eval_metric_ops=eval_metric_ops)
    
#     config = tf.ConfigProto()
#     config.gpu_options.allow_growth=True
#     sess = tf.Session(config=config)
#     sess.run(tf.global_variables_initializer())
#     return sess.run(pool2, feed_dict =
#                     {input_data: features,
#                      input_
#                     labels: labels})



#print(cnn_4_cifar(train_data[0:2], train_label[0:2]))

# config = tf.ConfigProto()
# config.gpu_options.allow_growth=True
# sess = tf.Session(config=config)

In [18]:
###### estimator ######
config1 = tf.estimator.RunConfig()
config1 = config1.replace(log_step_count_steps = 1000, save_summary_steps = 1000)


cifar_classifier = tf.estimator.Estimator(
    model_fn = cnn_4_cifar,
    model_dir="./tmp/cifar_cnn_lrn20_model",
    config = config1
)

###### log ######

tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

###### training ######
# train_data_tf = tf.convert_to_tensor(
#     value = train_data,
#     dtype = tf.float32
# )
print(tf.shape(train_data.astype(np.float32)))

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data.astype(np.float32)},
    y=train_label,
    batch_size=100,
    num_epochs=None,
    shuffle=True)



eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": val_data.astype(np.float32)},
    y=val_label,
    num_epochs=1,
    shuffle=False)

for i in range(50):
    cifar_classifier.train(
        input_fn=train_input_fn,
        steps = 1000
    )
# cifar_classifier.evaluate(input_fn=eval_input_fn)


    # hooks = [logging_hook]



INFO:tensorflow:Using config: {'_model_dir': './tmp/cifar_cnn_lrn20_model', '_tf_random_seed': 1, '_save_summary_steps': 1000, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 1000}
Tensor("Shape_6:0", shape=(2,), dtype=int32)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./tmp/cifar_cnn_lrn20_model/model.ckpt.
INFO:tensorflow:loss = 2.29133, step = 1
INFO:tensorflow:loss = 2.29919, step = 101 (0.823 sec)
INFO:tensorflow:loss = 2.25525, step = 201 (0.800 sec)
INFO:tensorflow:loss = 2.09982, step = 301 (0.799 sec)
INFO:tensorflow:loss = 2.01973, step = 401 (0.799 sec)
INFO:tensorflow:loss = 1.898, step = 501 (0.799 sec)
INFO:tensorflow:loss = 1.86892, step = 601 (0.799 sec)
INFO:tensorflow:loss = 1.83759, step = 701 (0.793 sec)
INFO:tensorflow:loss = 1.86383, step = 801 (0.812 sec)
INFO:tensorflow:loss = 1.89685,

INFO:tensorflow:Saving checkpoints for 9000 into ./tmp/cifar_cnn_lrn20_model/model.ckpt.
INFO:tensorflow:Loss for final step: 1.00488.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./tmp/cifar_cnn_lrn20_model/model.ckpt-9000
INFO:tensorflow:Saving checkpoints for 9001 into ./tmp/cifar_cnn_lrn20_model/model.ckpt.
INFO:tensorflow:loss = 0.995814, step = 9001
INFO:tensorflow:loss = 0.831537, step = 9101 (0.808 sec)
INFO:tensorflow:loss = 1.03191, step = 9201 (0.793 sec)
INFO:tensorflow:loss = 1.03192, step = 9301 (0.796 sec)
INFO:tensorflow:loss = 1.264, step = 9401 (0.796 sec)
INFO:tensorflow:loss = 1.04375, step = 9501 (0.799 sec)
INFO:tensorflow:loss = 0.99737, step = 9601 (0.795 sec)
INFO:tensorflow:loss = 0.971318, step = 9701 (0.796 sec)
INFO:tensorflow:loss = 0.897739, step = 9801 (0.797 sec)
INFO:tensorflow:loss = 0.94995, step = 9901 (0.795 sec)
INFO:tensorflow:Saving checkpoints for 10000 into ./tmp/cifar_cnn_lrn20_model/model.ckpt.
INFO:t

INFO:tensorflow:loss = 0.863547, step = 17901 (0.792 sec)
INFO:tensorflow:Saving checkpoints for 18000 into ./tmp/cifar_cnn_lrn20_model/model.ckpt.
INFO:tensorflow:Loss for final step: 0.744986.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./tmp/cifar_cnn_lrn20_model/model.ckpt-18000
INFO:tensorflow:Saving checkpoints for 18001 into ./tmp/cifar_cnn_lrn20_model/model.ckpt.
INFO:tensorflow:loss = 0.780996, step = 18001
INFO:tensorflow:loss = 0.731762, step = 18101 (0.810 sec)
INFO:tensorflow:loss = 0.706905, step = 18201 (0.801 sec)
INFO:tensorflow:loss = 0.637077, step = 18301 (0.789 sec)
INFO:tensorflow:loss = 0.955817, step = 18401 (0.794 sec)
INFO:tensorflow:loss = 0.687889, step = 18501 (0.793 sec)
INFO:tensorflow:loss = 0.807731, step = 18601 (0.802 sec)
INFO:tensorflow:loss = 0.634273, step = 18701 (0.796 sec)
INFO:tensorflow:loss = 0.863277, step = 18801 (0.795 sec)
INFO:tensorflow:loss = 1.09094, step = 18901 (0.803 sec)
INFO:tensorflow:S

INFO:tensorflow:loss = 0.515162, step = 26701 (0.798 sec)
INFO:tensorflow:loss = 0.778704, step = 26801 (0.803 sec)
INFO:tensorflow:loss = 0.631435, step = 26901 (0.800 sec)
INFO:tensorflow:Saving checkpoints for 27000 into ./tmp/cifar_cnn_lrn20_model/model.ckpt.
INFO:tensorflow:Loss for final step: 0.542412.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./tmp/cifar_cnn_lrn20_model/model.ckpt-27000
INFO:tensorflow:Saving checkpoints for 27001 into ./tmp/cifar_cnn_lrn20_model/model.ckpt.
INFO:tensorflow:loss = 0.603677, step = 27001
INFO:tensorflow:loss = 0.699519, step = 27101 (0.811 sec)
INFO:tensorflow:loss = 0.629185, step = 27201 (0.795 sec)
INFO:tensorflow:loss = 0.646367, step = 27301 (0.793 sec)
INFO:tensorflow:loss = 0.604841, step = 27401 (0.790 sec)
INFO:tensorflow:loss = 0.742728, step = 27501 (0.791 sec)
INFO:tensorflow:loss = 0.625385, step = 27601 (0.798 sec)
INFO:tensorflow:loss = 0.77288, step = 27701 (0.794 sec)
INFO:tensorflow:l

INFO:tensorflow:loss = 0.430815, step = 35501 (0.798 sec)
INFO:tensorflow:loss = 0.508049, step = 35601 (0.803 sec)
INFO:tensorflow:loss = 0.496433, step = 35701 (0.805 sec)
INFO:tensorflow:loss = 0.540015, step = 35801 (0.796 sec)
INFO:tensorflow:loss = 0.493295, step = 35901 (0.799 sec)
INFO:tensorflow:Saving checkpoints for 36000 into ./tmp/cifar_cnn_lrn20_model/model.ckpt.
INFO:tensorflow:Loss for final step: 0.452331.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./tmp/cifar_cnn_lrn20_model/model.ckpt-36000
INFO:tensorflow:Saving checkpoints for 36001 into ./tmp/cifar_cnn_lrn20_model/model.ckpt.
INFO:tensorflow:loss = 0.5587, step = 36001
INFO:tensorflow:loss = 0.49307, step = 36101 (0.814 sec)
INFO:tensorflow:loss = 0.389464, step = 36201 (0.790 sec)
INFO:tensorflow:loss = 0.430151, step = 36301 (0.799 sec)
INFO:tensorflow:loss = 0.407138, step = 36401 (0.792 sec)
INFO:tensorflow:loss = 0.507326, step = 36501 (0.801 sec)
INFO:tensorflow:los

INFO:tensorflow:loss = 0.435237, step = 44301 (0.792 sec)
INFO:tensorflow:loss = 0.397795, step = 44401 (0.795 sec)
INFO:tensorflow:loss = 0.306546, step = 44501 (0.800 sec)
INFO:tensorflow:loss = 0.332274, step = 44601 (0.801 sec)
INFO:tensorflow:loss = 0.360898, step = 44701 (0.793 sec)
INFO:tensorflow:loss = 0.431138, step = 44801 (0.795 sec)
INFO:tensorflow:loss = 0.339182, step = 44901 (0.802 sec)
INFO:tensorflow:Saving checkpoints for 45000 into ./tmp/cifar_cnn_lrn20_model/model.ckpt.
INFO:tensorflow:Loss for final step: 0.440124.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./tmp/cifar_cnn_lrn20_model/model.ckpt-45000
INFO:tensorflow:Saving checkpoints for 45001 into ./tmp/cifar_cnn_lrn20_model/model.ckpt.
INFO:tensorflow:loss = 0.411156, step = 45001
INFO:tensorflow:loss = 0.343602, step = 45101 (0.812 sec)
INFO:tensorflow:loss = 0.403396, step = 45201 (0.792 sec)
INFO:tensorflow:loss = 0.367066, step = 45301 (0.798 sec)
INFO:tensorflow:

NameError: name 'cdcifar_classifier' is not defined

In [19]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": val_data.astype(np.float32)},
    y=val_label,
    num_epochs=1,
    shuffle=False)
eval_results = cifar_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)


INFO:tensorflow:Starting evaluation at 2018-02-12-00:40:33
INFO:tensorflow:Restoring parameters from ./tmp/cifar_cnn_lrn20_model/model.ckpt-50000
INFO:tensorflow:Finished evaluation at 2018-02-12-00:40:34
INFO:tensorflow:Saving dict for global step 50000: accuracy = 0.701, global_step = 50000, loss = 1.14438
{'accuracy': 0.70099998, 'loss': 1.1443821, 'global_step': 50000}


In [20]:
from cnn_jupyter_tensorboard import show_graph
tf.reset_default_graph()
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
with tf.Session(config=config) as sess:
    saver = tf.train.import_meta_graph(
        './tmp/cifar_cnn_lrn20_model/model.ckpt-50000.meta')
    graph = tf.get_default_graph()
    show_graph(graph)